In [1]:
"""
Hospital Management System that demonstrates advanced encapsulation concepts with multiple interconnected 
classes, complex validation, and sophisticated access control.
"""

'\nHospital Management System that demonstrates advanced encapsulation concepts with multiple interconnected \nclasses, complex validation, and sophisticated access control.\n'

In [2]:
import hashlib
import uuid
from datetime import datetime, timedelta
from enum import Enum
from typing import Dict, List, Optional
import re

class AccessLevel(Enum):
    """Access levels for different user types"""
    PATIENT = 1
    NURSE = 2
    DOCTOR = 3
    ADMIN = 4
    SYSTEM = 5

class BloodType(Enum):
    """Blood type enumeration"""
    A_POS = "A+"
    A_NEG = "A-"
    B_POS = "B+"
    B_NEG = "B-"
    AB_POS = "AB+"
    AB_NEG = "AB-"
    O_POS = "O+"
    O_NEG = "O-"

class MedicalRecord:
    """Highly encapsulated medical record with strict access control"""
    
    def __init__(self, patient_id: str):
        # Private patient identifier
        self.__patient_id = patient_id
        
        # Ultra-sensitive medical data (private)
        self.__medical_history: List[Dict] = []
        self.__prescriptions: List[Dict] = []
        self.__lab_results: List[Dict] = []
        self.__diagnoses: List[Dict] = []
        self.__allergies: List[str] = []
        self.__blood_type: Optional[BloodType] = None
        
        # Audit trail for access control
        self.__access_log: List[Dict] = []
        self.__last_accessed = None
        self.__created_timestamp = datetime.now()
        
        # Security tokens for different access levels
        self.__doctor_access_tokens: Dict[str, datetime] = {}
        self.__nurse_access_tokens: Dict[str, datetime] = {}

    def __log_access(self, accessor_id: str, access_level: AccessLevel, action: str, data_type: str = ""):
        """Private method to log all access attempts"""
        log_entry = {
            'timestamp': datetime.now(),
            'accessor_id': accessor_id,
            'access_level': access_level.name,
            'action': action,
            'data_type': data_type,
            'ip_simulation': f"192.168.1.{hash(accessor_id) % 255}"
        }
        self.__access_log.append(log_entry)
        self.__last_accessed = datetime.now()

    def __verify_access_token(self, accessor_id: str, token: str, required_level: AccessLevel) -> bool:
        """Private method to verify access tokens"""
        if required_level == AccessLevel.DOCTOR:
            if accessor_id in self.__doctor_access_tokens:
                # Token expires after 8 hours
                if datetime.now() - self.__doctor_access_tokens[accessor_id] < timedelta(hours=8):
                    expected_token = self.__generate_access_token(accessor_id, required_level)
                    return token == expected_token
        elif required_level == AccessLevel.NURSE:
            if accessor_id in self.__nurse_access_tokens:
                # Token expires after 4 hours
                if datetime.now() - self.__nurse_access_tokens[accessor_id] < timedelta(hours=4):
                    expected_token = self.__generate_access_token(accessor_id, required_level)
                    return token == expected_token
        return False

    def __generate_access_token(self, accessor_id: str, access_level: AccessLevel) -> str:
        """Private method to generate secure access tokens"""
        secret_key = f"{self.__patient_id}_{accessor_id}_{access_level.name}_{datetime.now().date()}"
        return hashlib.sha256(secret_key.encode()).hexdigest()[:16]

    # Property for patient ID (read-only)
    @property
    def patient_id(self) -> str:
        """Get patient ID - read only"""
        return self.__patient_id

    # Property for last accessed time
    @property
    def last_accessed(self) -> Optional[datetime]:
        """Get when record was last accessed"""
        return self.__last_accessed

    # Property for creation timestamp
    @property
    def created_date(self) -> datetime:
        """Get when record was created"""
        return self.__created_timestamp

    def grant_doctor_access(self, doctor_id: str, admin_authorization: str) -> str:
        """Grant doctor access with admin authorization"""
        if not self.__verify_admin_authorization(admin_authorization):
            raise PermissionError("Invalid admin authorization for doctor access")
        
        self.__doctor_access_tokens[doctor_id] = datetime.now()
        token = self.__generate_access_token(doctor_id, AccessLevel.DOCTOR)
        self.__log_access(doctor_id, AccessLevel.DOCTOR, "ACCESS_GRANTED", "FULL_RECORD")
        return token

    def grant_nurse_access(self, nurse_id: str, doctor_authorization: str) -> str:
        """Grant nurse access with doctor authorization"""
        if not self.__verify_doctor_authorization(doctor_authorization):
            raise PermissionError("Invalid doctor authorization for nurse access")
        
        self.__nurse_access_tokens[nurse_id] = datetime.now()
        token = self.__generate_access_token(nurse_id, AccessLevel.NURSE)
        self.__log_access(nurse_id, AccessLevel.NURSE, "ACCESS_GRANTED", "LIMITED_RECORD")
        return token

    def __verify_admin_authorization(self, auth_code: str) -> bool:
        """Private method to verify admin authorization"""
        expected_code = hashlib.md5(f"ADMIN_{self.__patient_id}_{datetime.now().date()}".encode()).hexdigest()[:8]
        return auth_code == expected_code

    def __verify_doctor_authorization(self, auth_code: str) -> bool:
        """Private method to verify doctor authorization"""
        # Simplified doctor authorization check
        return auth_code.startswith("DOC_") and len(auth_code) == 12

    # Encapsulated method to add medical history
    def add_medical_history(self, accessor_id: str, token: str, entry: Dict) -> str:
        """Add medical history entry with proper authorization"""
        if not self.__verify_access_token(accessor_id, token, AccessLevel.DOCTOR):
            self.__log_access(accessor_id, AccessLevel.DOCTOR, "ACCESS_DENIED", "MEDICAL_HISTORY")
            raise PermissionError("Invalid doctor credentials for adding medical history")

        # Validate entry structure
        required_fields = ['date', 'condition', 'description', 'severity']
        if not all(field in entry for field in required_fields):
            raise ValueError(f"Medical history entry must contain: {required_fields}")

        # Add timestamp and doctor ID to entry
        entry['added_by'] = accessor_id
        entry['added_timestamp'] = datetime.now()
        entry['entry_id'] = str(uuid.uuid4())

        self.__medical_history.append(entry)
        self.__log_access(accessor_id, AccessLevel.DOCTOR, "DATA_ADDED", "MEDICAL_HISTORY")
        return entry['entry_id']

    # Property decorator with complex access control
    @property
    def medical_history(self) -> str:
        """Medical history requires proper token-based access"""
        return "Access denied. Use get_medical_history() with proper credentials."

    def get_medical_history(self, accessor_id: str, token: str, access_level: AccessLevel) -> List[Dict]:
        """Get medical history with role-based access control"""
        if access_level == AccessLevel.DOCTOR:
            if not self.__verify_access_token(accessor_id, token, AccessLevel.DOCTOR):
                self.__log_access(accessor_id, access_level, "ACCESS_DENIED", "MEDICAL_HISTORY")
                raise PermissionError("Invalid doctor credentials")
            
            self.__log_access(accessor_id, access_level, "DATA_ACCESSED", "MEDICAL_HISTORY_FULL")
            return self.__medical_history  # Full access

        elif access_level == AccessLevel.NURSE:
            if not self.__verify_access_token(accessor_id, token, AccessLevel.NURSE):
                self.__log_access(accessor_id, access_level, "ACCESS_DENIED", "MEDICAL_HISTORY")
                raise PermissionError("Invalid nurse credentials")
            
            # Nurses get limited access (last 30 days only)
            cutoff_date = datetime.now() - timedelta(days=30)
            recent_history = [
                {k: v for k, v in entry.items() if k not in ['added_by', 'entry_id']}
                for entry in self.__medical_history
                if entry.get('date', datetime.min) > cutoff_date
            ]
            
            self.__log_access(accessor_id, access_level, "DATA_ACCESSED", "MEDICAL_HISTORY_LIMITED")
            return recent_history

        else:
            self.__log_access(accessor_id, access_level, "ACCESS_DENIED", "MEDICAL_HISTORY")
            raise PermissionError("Insufficient access level for medical history")

    # Blood type with sophisticated validation
    @property
    def blood_type(self) -> str:
        """Get blood type (basic access)"""
        return self.__blood_type.value if self.__blood_type else "Not specified"

    @blood_type.setter
    def blood_type(self, value):
        """Set blood type with validation - requires doctor access"""
        raise AttributeError("Blood type cannot be set directly. Use set_blood_type() with proper credentials.")

    def set_blood_type(self, accessor_id: str, token: str, blood_type_value: str) -> str:
        """Set blood type with doctor authorization and validation"""
        if not self.__verify_access_token(accessor_id, token, AccessLevel.DOCTOR):
            self.__log_access(accessor_id, AccessLevel.DOCTOR, "ACCESS_DENIED", "BLOOD_TYPE_UPDATE")
            raise PermissionError("Only doctors can set blood type")

        # Validate blood type
        try:
            blood_type_enum = BloodType(blood_type_value)
        except ValueError:
            valid_types = [bt.value for bt in BloodType]
            raise ValueError(f"Invalid blood type. Must be one of: {valid_types}")

        old_type = self.__blood_type
        self.__blood_type = blood_type_enum
        
        self.__log_access(accessor_id, AccessLevel.DOCTOR, "DATA_UPDATED", "BLOOD_TYPE")
        return f"Blood type updated from {old_type.value if old_type else 'None'} to {blood_type_value}"

    # Prescription management with complex validation
    def add_prescription(self, accessor_id: str, token: str, prescription: Dict) -> str:
        """Add prescription with doctor authorization and drug interaction checking"""
        if not self.__verify_access_token(accessor_id, token, AccessLevel.DOCTOR):
            self.__log_access(accessor_id, AccessLevel.DOCTOR, "ACCESS_DENIED", "PRESCRIPTION_ADD")
            raise PermissionError("Only doctors can prescribe medications")

        # Validate prescription structure
        required_fields = ['medication', 'dosage', 'frequency', 'duration', 'instructions']
        if not all(field in prescription for field in required_fields):
            raise ValueError(f"Prescription must contain: {required_fields}")

        # Check for drug interactions (simplified)
        if self.__check_drug_interactions(prescription['medication']):
            raise ValueError(f"Drug interaction detected with {prescription['medication']}")

        # Add metadata
        prescription['prescribed_by'] = accessor_id
        prescription['prescribed_date'] = datetime.now()
        prescription['prescription_id'] = str(uuid.uuid4())
        prescription['status'] = 'ACTIVE'

        self.__prescriptions.append(prescription)
        self.__log_access(accessor_id, AccessLevel.DOCTOR, "PRESCRIPTION_ADDED", prescription['medication'])
        return prescription['prescription_id']

    def __check_drug_interactions(self, new_medication: str) -> bool:
        """Private method to check for drug interactions"""
        # Simplified drug interaction checking
        dangerous_combinations = {
            'warfarin': ['aspirin', 'ibuprofen'],
            'aspirin': ['warfarin'],
            'metformin': ['alcohol'],
        }
        
        current_medications = [p['medication'].lower() for p in self.__prescriptions if p['status'] == 'ACTIVE']
        new_med_lower = new_medication.lower()
        
        if new_med_lower in dangerous_combinations:
            for current_med in current_medications:
                if current_med in dangerous_combinations[new_med_lower]:
                    return True
        
        return False

    # Audit trail access (admin only)
    def get_access_log(self, admin_id: str, admin_authorization: str) -> List[Dict]:
        """Get access log with admin authorization"""
        if not self.__verify_admin_authorization(admin_authorization):
            raise PermissionError("Invalid admin authorization for access log")

        self.__log_access(admin_id, AccessLevel.ADMIN, "AUDIT_LOG_ACCESSED", "FULL_LOG")
        return self.__access_log.copy()  # Return copy to prevent modification

    # Summary method with different levels of detail
    def get_summary(self, accessor_id: str, token: str, access_level: AccessLevel) -> Dict:
        """Get patient summary based on access level"""
        summary = {
            'patient_id': self.__patient_id,
            'last_accessed': self.__last_accessed,
            'created_date': self.__created_timestamp
        }

        if access_level == AccessLevel.DOCTOR and self.__verify_access_token(accessor_id, token, AccessLevel.DOCTOR):
            summary.update({
                'total_medical_entries': len(self.__medical_history),
                'active_prescriptions': len([p for p in self.__prescriptions if p['status'] == 'ACTIVE']),
                'blood_type': self.blood_type,
                'known_allergies': len(self.__allergies),
                'lab_results_count': len(self.__lab_results)
            })
            self.__log_access(accessor_id, access_level, "SUMMARY_ACCESSED", "DETAILED")

        elif access_level == AccessLevel.NURSE and self.__verify_access_token(accessor_id, token, AccessLevel.NURSE):
            summary.update({
                'active_prescriptions': len([p for p in self.__prescriptions if p['status'] == 'ACTIVE']),
                'blood_type': self.blood_type,
                'recent_entries': len([h for h in self.__medical_history 
                                     if (datetime.now() - h.get('date', datetime.min)).days <= 7])
            })
            self.__log_access(accessor_id, access_level, "SUMMARY_ACCESSED", "LIMITED")

        else:
            self.__log_access(accessor_id, access_level, "ACCESS_DENIED", "SUMMARY")
            raise PermissionError("Invalid credentials for patient summary")

        return summary


class Hospital:
    """Hospital management system with sophisticated encapsulation"""
    
    def __init__(self, name: str):
        self.name = name
        
        # Private hospital data
        self.__medical_records: Dict[str, MedicalRecord] = {}
        self.__staff_credentials: Dict[str, Dict] = {}
        self.__admin_keys: Dict[str, datetime] = {}
        
        # Protected hospital statistics
        self._total_patients = 0
        self._total_staff = 0
        
        # System configuration (private)
        self.__system_config = {
            'max_patients': 10000,
            'security_level': 'HIGH',
            'audit_retention_days': 365
        }

    def __generate_admin_authorization(self, patient_id: str) -> str:
        """Private method to generate admin authorization codes"""
        return hashlib.md5(f"ADMIN_{patient_id}_{datetime.now().date()}".encode()).hexdigest()[:8]

    def register_patient(self, patient_id: str, admin_id: str, admin_key: str) -> MedicalRecord:
        """Register new patient with admin authorization"""
        if not self.__verify_admin_key(admin_id, admin_key):
            raise PermissionError("Invalid admin credentials for patient registration")

        if patient_id in self.__medical_records:
            raise ValueError(f"Patient {patient_id} already registered")

        if len(self.__medical_records) >= self.__system_config['max_patients']:
            raise ValueError("Hospital at maximum patient capacity")

        # Create new medical record
        medical_record = MedicalRecord(patient_id)
        self.__medical_records[patient_id] = medical_record
        self._total_patients += 1

        return medical_record

    def __verify_admin_key(self, admin_id: str, admin_key: str) -> bool:
        """Private method to verify admin keys"""
        if admin_id in self.__admin_keys:
            # Admin key expires after 24 hours
            if datetime.now() - self.__admin_keys[admin_id] < timedelta(hours=24):
                expected_key = hashlib.sha256(f"{admin_id}_ADMIN_{datetime.now().date()}".encode()).hexdigest()[:12]
                return admin_key == expected_key
        return False

    def register_admin(self, admin_id: str, master_password: str) -> str:
        """Register admin with master password"""
        if master_password != "HOSPITAL_MASTER_2024":  # In real system, this would be much more secure
            raise PermissionError("Invalid master password")

        self.__admin_keys[admin_id] = datetime.now()
        admin_key = hashlib.sha256(f"{admin_id}_ADMIN_{datetime.now().date()}".encode()).hexdigest()[:12]
        return admin_key

    def get_patient_record(self, patient_id: str, admin_id: str, admin_key: str) -> MedicalRecord:
        """Get patient record with admin verification"""
        if not self.__verify_admin_key(admin_id, admin_key):
            raise PermissionError("Invalid admin credentials")

        if patient_id not in self.__medical_records:
            raise ValueError(f"Patient {patient_id} not found")

        return self.__medical_records[patient_id]

    def authorize_doctor_access(self, patient_id: str, doctor_id: str, admin_id: str, admin_key: str) -> str:
        """Authorize doctor access to patient record"""
        if not self.__verify_admin_key(admin_id, admin_key):
            raise PermissionError("Invalid admin credentials")

        if patient_id not in self.__medical_records:
            raise ValueError(f"Patient {patient_id} not found")

        admin_auth = self.__generate_admin_authorization(patient_id)
        return self.__medical_records[patient_id].grant_doctor_access(doctor_id, admin_auth)

    # Property for hospital statistics
    @property
    def patient_count(self) -> int:
        """Get current patient count"""
        return self._total_patients

    @property
    def system_status(self) -> Dict:
        """Get system status (limited information)"""
        return {
            'name': self.name,
            'patient_count': self._total_patients,
            'system_security': self.__system_config['security_level'],
            'status': 'OPERATIONAL'
        }


# Demonstration
def demonstrate_complex_encapsulation():
    print("=== COMPLEX HOSPITAL ENCAPSULATION DEMONSTRATION ===\n")
    
    # Initialize hospital
    hospital = Hospital("St. Mary's General Hospital")
    print(f"Hospital initialized: {hospital.name}")
    print(f"System status: {hospital.system_status}\n")
    
    # Register admin
    print("1. Admin Registration")
    try:
        admin_key = hospital.register_admin("admin001", "HOSPITAL_MASTER_2024")
        print(f"Admin registered successfully. Key: {admin_key}")
    except PermissionError as e:
        print(f"Admin registration failed: {e}")
    
    # Register patient
    print(f"\n2. Patient Registration")
    try:
        patient_record = hospital.register_patient("P001", "admin001", admin_key)
        print(f"Patient P001 registered successfully")
        print(f"Record created: {patient_record.created_date}")
    except (PermissionError, ValueError) as e:
        print(f"Patient registration failed: {e}")
    
    # Authorize doctor access
    print(f"\n3. Doctor Authorization")
    try:
        doctor_token = hospital.authorize_doctor_access("P001", "DR001", "admin001", admin_key)
        print(f"Doctor DR001 authorized. Token: {doctor_token[:8]}...")
    except (PermissionError, ValueError) as e:
        print(f"Doctor authorization failed: {e}")
    
    # Set blood type (doctor only)
    print(f"\n4. Setting Blood Type (Doctor Access Required)")
    try:
        result = patient_record.set_blood_type("DR001", doctor_token, "A+")
        print(f"Blood type set: {result}")
        print(f"Current blood type: {patient_record.blood_type}")
    except (PermissionError, ValueError) as e:
        print(f"Blood type setting failed: {e}")
    
    # Add medical history
    print(f"\n5. Adding Medical History (Doctor Access)")
    medical_entry = {
        'date': datetime.now(),
        'condition': 'Hypertension',
        'description': 'High blood pressure diagnosed during routine checkup',
        'severity': 'Moderate'
    }
    
    try:
        entry_id = patient_record.add_medical_history("DR001", doctor_token, medical_entry)
        print(f"Medical history added. Entry ID: {entry_id}")
    except (PermissionError, ValueError) as e:
        print(f"Medical history addition failed: {e}")
    
    # Add prescription with drug interaction checking
    print(f"\n6. Adding Prescription (with Drug Interaction Check)")
    prescription = {
        'medication': 'Lisinopril',
        'dosage': '10mg',
        'frequency': 'Once daily',
        'duration': '30 days',
        'instructions': 'Take with food'
    }
    
    try:
        prescription_id = patient_record.add_prescription("DR001", doctor_token, prescription)
        print(f"Prescription added. ID: {prescription_id}")
    except (PermissionError, ValueError) as e:
        print(f"Prescription addition failed: {e}")
    
    # Grant nurse access
    print(f"\n7. Granting Nurse Access")
    try:
        nurse_token = patient_record.grant_nurse_access("RN001", "DOC_AUTH123")
        print(f"Nurse RN001 authorized. Token: {nurse_token[:8]}...")
    except PermissionError as e:
        print(f"Nurse authorization failed: {e}")
    
    # Compare access levels
    print(f"\n8. Comparing Access Levels")
    
    # Doctor access
    try:
        doctor_summary = patient_record.get_summary("DR001", doctor_token, AccessLevel.DOCTOR)
        print(f"Doctor summary: {doctor_summary}")
    except PermissionError as e:
        print(f"Doctor summary failed: {e}")
    
    # Nurse access
    try:
        nurse_summary = patient_record.get_summary("RN001", nurse_token, AccessLevel.NURSE)
        print(f"Nurse summary: {nurse_summary}")
    except PermissionError as e:
        print(f"Nurse summary failed: {e}")
    
    # Unauthorized access attempts
    print(f"\n9. Testing Security - Unauthorized Access Attempts")
    
    # Try to access without proper token
    try:
        patient_record.get_medical_history("HACKER001", "fake_token", AccessLevel.DOCTOR)
        print("Security breach!")
    except PermissionError as e:
        print(f"Security working: {e}")
    
    # Try to set blood type without authorization
    try:
        patient_record.set_blood_type("NURSE001", "fake_token", "B+")
        print("Security breach!")
    except PermissionError as e:
        print(f"Security working: {e}")
    
    # Admin audit trail
    print(f"\n10. Admin Audit Trail")
    try:
        admin_auth = hospital._Hospital__generate_admin_authorization("P001")
        audit_log = patient_record.get_access_log("admin001", admin_auth)
        print(f"Total access attempts logged: {len(audit_log)}")
        print("Recent access attempts:")
        for entry in audit_log[-3:]:
            print(f"  {entry['timestamp'].strftime('%H:%M:%S')}: {entry['accessor_id']} - {entry['action']} - {entry['data_type']}")
    except PermissionError as e:
        print(f"Audit access failed: {e}")
    
    print(f"\n=== ENCAPSULATION FEATURES DEMONSTRATED ===")
    print("✓ Multi-layer access control with tokens")
    print("✓ Role-based permissions (Admin, Doctor, Nurse)")
    print("✓ Time-based token expiration")
    print("✓ Comprehensive audit logging")
    print("✓ Data validation and sanitization")
    print("✓ Drug interaction checking")
    print("✓ Secure property decorators")
    print("✓ Private method protection")
    print("✓ Complex authorization chains")


if __name__ == "__main__":
    demonstrate_complex_encapsulation()

=== COMPLEX HOSPITAL ENCAPSULATION DEMONSTRATION ===



Hospital initialized: St. Mary's General Hospital
System status: {'name': "St. Mary's General Hospital", 'patient_count': 0, 'system_security': 'HIGH', 'status': 'OPERATIONAL'}

1. Admin Registration
Admin registered successfully. Key: cef5371d8093

2. Patient Registration
Patient P001 registered successfully
Record created: 2025-06-18 21:31:48.642096

3. Doctor Authorization
Doctor DR001 authorized. Token: 4b07bf43...

4. Setting Blood Type (Doctor Access Required)
Blood type set: Blood type updated from None to A+
Current blood type: A+

5. Adding Medical History (Doctor Access)
Medical history added. Entry ID: 450fe12b-ffb7-40c9-bbf0-0b2ad1e52779

6. Adding Prescription (with Drug Interaction Check)
Prescription added. ID: 5b84fd12-726d-4692-9f16-b57602d52a67

7. Granting Nurse Access
Nurse authorization failed: Invalid doctor authorization for nurse access

8. Comparing Access Levels
Doctor summary: {'patient_id': 'P001', 'last_accessed': datetime.datetime(2025, 6, 18, 21, 31, 48,

UnboundLocalError: local variable 'nurse_token' referenced before assignment